In [1]:
pip install reservoirpy torch torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 kB 4.0 MB/s eta 0:00:00


In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from reservoirpy.nodes import Reservoir
from torchvision import datasets, transforms
from tqdm import trange

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

Device: cpu


In [3]:
# Compute ReservoirPy states
def compute_reservoir_states(reservoir, X_images):
    """
    X_images: numpy array shape [N_samples, 28, 28]
    Produce for each sample a flattened vector of states: [N_units * T]
    Return: states numpy array shape [N_samples, N_units * T]
    """
    n_samples = X_images.shape[0]
    T, in_dim = X_images.shape[1], X_images.shape[2]
    # collect states per sample
    states_out = []
    for i in range(n_samples):
        seq = X_images[i]  # shape [T, in_dim]
        # reservoir.run expects shape (T, in_dim)
        out = reservoir.run(seq)  # returns shape [T, n_units] (states per timestep)
        # flatten timesteps and units into single vector
        flat = out.reshape(-1)  # size n_units * T
        states_out.append(flat)
    return np.vstack(states_out).astype(np.float32)  # shape [n_samples, n_units*T]


In [4]:
# Load MNIST format as sequences
def load_mnist_seq(train=True):
    # Download raw MNIST images as [N, 28, 28] floats in [0,1]
    transform = transforms.Compose([transforms.ToTensor()])
    train_set = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
    test_set  = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

    X_tr = train_set.data.numpy().astype(np.float32) / 255.0  # [60000,28,28]
    Y_tr = np.eye(10)[train_set.targets.numpy()]             # one-hot [60000,10]
    X_te = test_set.data.numpy().astype(np.float32) / 255.0   # [10000,28,28]
    Y_te = np.eye(10)[test_set.targets.numpy()]
    return X_tr, Y_tr.astype(np.float32), X_te, Y_te.astype(np.float32)

In [5]:
# SpaRCe model
class SpaRCeModel(nn.Module):
    def __init__(self, in_features, n_classes, theta_i_init=None, theta_g=None):
        """
        in_features: size [N_units * T] - flattened reservoir states
        n_classes: number of output classes
        theta_i_init: numpy array shape (in_features,) initial theta_i
        theta_g: numpy array shape (in_features,) fixed global threshold
        """
        super().__init__()
        self.in_features = in_features
        self.n_classes = n_classes

        # linear readout
        self.readout = nn.Linear(in_features, n_classes, bias=False)

        # local threshold theta_i trainable per neuron
        if theta_i_init is None:
            theta_i_init = np.random.randn(in_features).astype(np.float32) / (in_features**0.5)
        self.theta_i = nn.Parameter(torch.from_numpy(theta_i_init).float())  # shape [in_features]

        # fixed global threshold theta_g (non-trainable)
        if theta_g is None:
            theta_g = np.zeros((in_features,), dtype=np.float32)
        self.register_buffer("theta_g", torch.from_numpy(theta_g).float())  # buffer, not parameter

    def forward(self, state):
        """
        state: tensor [batch, in_features] (raw reservoir states)
        returns logits [batch, n_classes] and state_sparse [batch, in_features]
        Activation: state_sparse = sign(state) * relu(abs(state) - theta_g - theta_i)
        """
        # broadcast thresholds
        th = (self.theta_g + self.theta_i).unsqueeze(0)  # [1, in_features]
        s_abs = torch.abs(state)
        s_thresh = F.relu(s_abs - th)  # [batch, in_features]
        state_sparse = torch.sign(state) * s_thresh
        logits = self.readout(state_sparse)
        return logits, state_sparse

In [6]:
# Training of ESN or SpaRCe
def train_models(States_tr, Y_tr, States_te, Y_te, States_val=None,
                 MODEL=1,            # 1 = SpaRCe, 2 = Standard ESN
                 Pns_list=[70],      # percentiles (only for SpaRCe)
                 alpha_list=[1e-3],  # learning rates (for Standard ESN can be list)
                 batch_size=128,
                 N_episodes=2000,
                 N_check=20,
                 device=device):
    """
    States_*: numpy arrays of shape [N_samples, in_features]
    Y_*: numpy arrays (one-hot vectors) of shape [N_samples, n_classes]
    Returns: Results_tr, Results_te, Results_val arrays
    """
    States_tr = States_tr.astype(np.float32)
    States_te = States_te.astype(np.float32)
    if States_val is not None:
        States_val = States_val.astype(np.float32)

    N_train = States_tr.shape[0]
    N_test  = States_te.shape[0]
    N_class = Y_tr.shape[1]
    in_features = States_tr.shape[1]

    # splits for evaluation: mirroring original code logic
    train_divide = 100
    test_divide  = 50
    val_divide   = 50

    N_train_d = int(np.floor(N_train / train_divide))
    N_test_d  = int(np.floor(N_test / test_divide))
    if States_val is not None:
        N_val = States_val.shape[0]
        N_val_d = int(np.floor(N_val / val_divide))
    else:
        N_val = 0
        N_val_d = 0

    # prepare PyTorch datasets for training (will sample random indices)
    train_dataset = TensorDataset(torch.from_numpy(States_tr), torch.from_numpy(Y_tr))
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=False)

    # prepare results containers
    if MODEL == 1:
        N_copies = len(Pns_list)
        Results_tr = np.zeros((N_copies, N_check, 3), dtype=np.float32)  # [error, accuracy, coding]
        Results_te = np.zeros((N_copies, N_check, 3), dtype=np.float32)
        Results_val= np.zeros((N_copies, N_check, 3), dtype=np.float32)
    else:
        N_copies = len(alpha_list)
        Results_tr = np.zeros((N_copies, N_check, 2), dtype=np.float32)  # [error, accuracy]
        Results_te = np.zeros((N_copies, N_check, 2), dtype=np.float32)
        Results_val= np.zeros((N_copies, N_check, 2), dtype=np.float32)

    # loss function: sigmoid cross entropy (BCEWithLogits for multi-label)
    criterion = nn.BCEWithLogitsLoss(reduction='mean')

    # function to evaluate a model on a dataset by splitting into smaller parts called 'divisions'
    def evaluate_model(model, States, Y, division):
        model.eval()
        total_loss = 0.0
        total_correct = 0
        total_count = 0
        coding_sum = 0.0
        n_parts = division
        part_size = int(np.floor(States.shape[0] / n_parts)) if n_parts>0 else States.shape[0]
        # iterate parts
        for part in range(n_parts):
            start = part * part_size
            end = (part + 1) * part_size
            Xp = torch.from_numpy(States[start:end]).to(device)
            Yp = torch.from_numpy(Y[start:end]).to(device)
            with torch.no_grad():
                logits, state_sparse = model(Xp)
                loss = criterion(logits, Yp)
                total_loss += loss.item() * (end - start)
                preds = torch.argmax(torch.sigmoid(logits), dim=1)
                labels = torch.argmax(Yp, dim=1)
                total_correct += (preds == labels).sum().item()
                total_count += (end - start)
                # coding level (fraction of non-zero activations)
                if hasattr(model, "theta_i"):
                    coding_sum += (state_sparse != 0).float().sum().item()
        # processing the rest
        rem = States.shape[0] - n_parts * part_size
        if rem > 0:
            start = n_parts * part_size
            Xp = torch.from_numpy(States[start:]).to(device)
            Yp = torch.from_numpy(Y[start:]).to(device)
            with torch.no_grad():
                logits, state_sparse = model(Xp)
                loss = criterion(logits, Yp)
                total_loss += loss.item() * rem
                preds = torch.argmax(torch.sigmoid(logits), dim=1)
                labels = torch.argmax(Yp, dim=1)
                total_correct += (preds == labels).sum().item()
                total_count += rem
                if hasattr(model, "theta_i"):
                    coding_sum += (state_sparse != 0).float().sum().item()
        avg_loss = total_loss / total_count
        acc = total_correct / total_count
        coding = None
        if hasattr(model, "theta_i"):
            coding = coding_sum / (total_count * (in_features / States.shape[1]) )  # simplified: fraction of non-zero elements per sample
            # compute coding fraction per-sample
            coding = coding_sum / (total_count * in_features)
        model.train()
        return avg_loss, acc, coding

    # MAIN LOOP: train separate models per copy
    for copy_idx in range(N_copies):
        print(f"\n=== Training copy {copy_idx+1}/{N_copies} ===")
        # build model and optimizers depending on MODEL
        # if model is SpaRCe
        if MODEL == 1:
            # compute theta_g_start for this copy (percentile across training states per feature)
            P = Pns_list[copy_idx]
            # states_tr of shape [N_train, in_features] -> compute percentile per feature
            theta_g = np.percentile(np.abs(States_tr), P, axis=0).astype(np.float32)  # shape [in_features,]
            theta_i_init = (np.random.randn(in_features).astype(np.float32) / max(1.0, in_features**0.5))
            model = SpaRCeModel(in_features, N_class, theta_i_init=theta_i_init, theta_g=theta_g).to(device)
            # optimizers: one for readout (output layer), one for theta_i (trainable neural thresholds)
            alpha = 1e-3 if len(alpha_list)==0 else alpha_list[0]
            opt_readout = torch.optim.Adam(model.readout.parameters(), lr=alpha)
            opt_theta   = torch.optim.Adam([model.theta_i], lr=alpha/10.0)
            # step both optimizers each batch
        else:
            # Standard ESN: no trainable theta_i, only linear readout
            model = nn.Linear(in_features, N_class, bias=False).to(device)
            # wrap into a small wrapper to reuse evaluation code expecting model(X)->(logits,state_sparse)
            class StdWrapper(nn.Module):
                def __init__(self, linear):
                    super().__init__()
                    self.linear = linear
                def forward(self, state):
                    logits = self.linear(state)
                    return logits, None
            model = StdWrapper(model)
            # pick learning rate for this copy
            alpha = alpha_list[copy_idx]
            opt_readout = torch.optim.Adam(model.parameters(), lr=alpha)
            opt_theta = None

        # training loop with checkpoints
        check_interval = max(1, int(np.round(N_episodes / N_check)))
        train_iter = trange(N_episodes, desc=f"Copy {copy_idx+1}")
        for n in train_iter:
            # one mini-batch update (sample random batch)
            batch_idx = np.random.randint(0, N_train, size=(batch_size,))
            batch_x = torch.from_numpy(States_tr[batch_idx]).to(device)
            batch_y = torch.from_numpy(Y_tr[batch_idx]).to(device)
            # forward
            logits, state_sparse = model(batch_x)
            loss = criterion(logits, batch_y)
            # backward and update
            opt_readout.zero_grad()
            if opt_theta is not None:
                opt_theta.zero_grad()
            loss.backward()
            opt_readout.step()
            if opt_theta is not None:
                opt_theta.step()

            # checkpoint evaluation
            if n % check_interval == 0:
                idx = int(n / check_interval)
                # evaluate on test/train/val as authors do
                # test
                if MODEL == 2:
                    loss_te, acc_te, _ = evaluate_model(model, States_te, Y_te, test_divide)
                    loss_tr, acc_tr, _ = evaluate_model(model, States_tr, Y_tr, train_divide)
                    if States_val is not None:
                        loss_val, acc_val, _ = evaluate_model(model, States_val, Y_val, val_divide)
                    else:
                        loss_val, acc_val = 0.0, 0.0
                    Results_te[copy_idx, idx, 0] = loss_te
                    Results_te[copy_idx, idx, 1] = acc_te
                    Results_tr[copy_idx, idx, 0] = loss_tr
                    Results_tr[copy_idx, idx, 1] = acc_tr
                    if States_val is not None:
                        Results_val[copy_idx, idx, 0] = loss_val
                        Results_val[copy_idx, idx, 1] = acc_val
                    print(f"[ESN] Iter {n} copy {copy_idx} TEST acc={acc_te:.4f} loss={loss_te:.4f}")
                else:
                    loss_te, acc_te, coding_te = evaluate_model(model, States_te, Y_te, test_divide)
                    loss_tr, acc_tr, coding_tr = evaluate_model(model, States_tr, Y_tr, train_divide)
                    if States_val is not None:
                        loss_val, acc_val, coding_val = evaluate_model(model, States_val, Y_val, val_divide)
                    else:
                        loss_val, acc_val, coding_val = 0.0, 0.0, 0.0
                    Results_te[copy_idx, idx, 0] = loss_te
                    Results_te[copy_idx, idx, 1] = acc_te
                    Results_te[copy_idx, idx, 2] = coding_te if coding_te is not None else 0.0
                    Results_tr[copy_idx, idx, 0] = loss_tr
                    Results_tr[copy_idx, idx, 1] = acc_tr
                    Results_tr[copy_idx, idx, 2] = coding_tr if coding_tr is not None else 0.0
                    if States_val is not None:
                        Results_val[copy_idx, idx, 0] = loss_val
                        Results_val[copy_idx, idx, 1] = acc_val
                        Results_val[copy_idx, idx, 2] = coding_val if coding_val is not None else 0.0
                    print(f"[SpaRCe] Iter {n} copy {copy_idx} TEST acc={acc_te:.4f} loss={loss_te:.4f} coding={coding_te:.4f}")

    return Results_tr, Results_te, Results_val


In [10]:
# running the whole program
if __name__ == "__main__":
    # Load MNIST and convert to sequences - columns
    X_tr, Y_tr, X_te, Y_te = load_mnist_seq()
    # reshape to [N, T, in_dim] where T=28 (timesteps), in_dim=28 (features per timestep)
    X_tr_seq = X_tr.reshape(-1, 28, 28)
    X_te_seq = X_te.reshape(-1, 28, 28)

    # build an ESN with reservoirpy
    # hyperparams (example)
    N_units = 300         # reservoir size
    sr = 0.97             # spectral radius
    input_scaling = 1.0   # gamma
    density = 0.05        # internal sparsity

    # create ReservoirPy Reservoir node
    res = Reservoir(units=N_units, sr=sr, input_scaling=input_scaling, input_connectivity=density, activation=np.tanh)

    # compute reservoir states
    print("Computing reservoir states for training set...")
    S_tr = compute_reservoir_states(res, X_tr_seq)  # shape [N_train, N_units*T]
    print("Computing reservoir states for test set...")
    S_te = compute_reservoir_states(res, X_te_seq)

    # example: train SpaRCe (MODEL=1) with two Pns (70% and 90%)
    Pns_list = [70]           # list of percentiles to test
    alpha_list = [1e-3]       # learning rate used for readout (SpaRCe uses alpha and alpha/10 for theta)
    Results_tr, Results_te, Results_val = train_models(S_tr, Y_tr, S_te, Y_te,
                                                       States_val=None,
                                                       MODEL=1,
                                                       Pns_list=Pns_list,
                                                       alpha_list=alpha_list,
                                                       batch_size=128,
                                                       N_episodes=1000,
                                                       N_check=10,
                                                       device=device)

    print("Training finished.")
    print("Results (test):", Results_te)

Computing reservoir states for training set...
Computing reservoir states for test set...

=== Training copy 1/1 ===


Copy 1:   1%|▏         | 13/1000 [00:03<03:03,  5.36it/s]

[SpaRCe] Iter 0 copy 0 TEST acc=0.2216 loss=0.6071 coding=0.3025


Copy 1:  12%|█▏        | 118/1000 [00:07<01:06, 13.24it/s]

[SpaRCe] Iter 100 copy 0 TEST acc=0.8740 loss=0.1144 coding=0.3124


...

[SpaRCe] Iter 800 copy 0 TEST acc=0.9397 loss=0.0488 coding=0.3409


Copy 1:  91%|█████████ | 911/1000 [00:41<00:05, 15.02it/s]

[SpaRCe] Iter 900 copy 0 TEST acc=0.9457 loss=0.0467 coding=0.3425


Copy 1: 100%|██████████| 1000/1000 [00:42<00:00, 23.56it/s]

Training finished.
Results (test): [[[0.607086   0.2216     0.30246806]
  [0.11438447 0.874      0.3123698 ]
  [0.08308282 0.9046     0.32051343]
  [0.07025962 0.9155     0.32620943]
  [0.06258667 0.924      0.33048436]
  [0.05782694 0.9278     0.33387926]
  [0.05438729 0.9325     0.33658767]
  [0.05115807 0.9382     0.33888468]
  [0.04875993 0.9397     0.34090406]
  [0.04669606 0.9457     0.34249923]]]
